# HW01

이름 : 박유나
학번 : 2021-12659 

# 1. preprocessing

## library 및 파일 불러오기

In [7]:
import nltk
import string
from gensim.models import Word2Vec
from tqdm import tqdm

In [8]:

with open("gutenberg.txt", "r") as f:
    data = f.read().splitlines()


## 불용어 및 문자 제거

In [9]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english') + list(string.punctuation)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YUNA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\YUNA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

nltk의 stopwords 및 punkt 모듈을 이용하여 불용어 목록을 만든다. 문장부호는 없애지 않을 수 있으나 향후 과제(단어 유사성 비교 및 벡터 임베팅 추출)에 의미적인 연관성을 갖고 있지 않다고 생각하기에 명확성을 위해 삭제하도록 하겠다.

## 토큰화

In [11]:
tokens = []
for line in data:
    words = nltk.word_tokenize(line.lower())
    tokens.append([word for word in words if word not in stopwords])

nltk의 word_tokenize()를 이용하여 토큰화하고 불용어 리스트에 있지 않은 딘어들만 tokens 리스트에 저장한다.

# 2. Train 


## create model

In [20]:
model_2 = Word2Vec(
    sg=1, #skip-gram
    min_count=2,#코퍼스에서 2번 미만으로 나온 단어는 생략
    vector_size=100, #vector dimension은 100, glove모델과 동일하게 100차원을 사용
    window=2, # window size는 2
    negative=5, # negative sampling은 5
)

In [19]:
model_5 = Word2Vec(
    sg=1, #skip-gram
    min_count=2,#코퍼스에서 2번 미만으로 나온 단어는 생략
    vector_size=100, #vector dimension은 100, glove모델과 동일하게 100차원을 사용
    window=5, # window size는 5
    negative=5, # negative sampling은 5
)

In [21]:
model_10 = Word2Vec(
    sg=1, #skip-gram
    min_count=2,#코퍼스에서 2번 미만으로 나온 단어는 생략
    vector_size=100, #vector dimension은 100, glove모델과 동일하게 100차원을 사용
    window=10, # window size는 10
    negative=5, # negative sampling은 5
)

skip-gram은 중심단어 1개로 주변단어 4개(window size는 4)를 예측하는 반면 cbow는 주변단어 4개로 중심단어 1개를 예측하기 때문에 skip-gram이 일반적으로 성능이 좋다고 알려져 있다. 그렇기에 skip-gram을 선택한다. 
window size나 vector size, negative sampling등이 커지면 당연히 성능은 좋아지겠으나 사용중인 노트북의 성능이 그다지 좋지 않으므로 적당한 수준으로 설정. window size에 대한 비교를 위해 window 파라미터가 각각 2, 5, 10인 model_2, model_5, model_10을 설정.

In [22]:
model_2.build_vocab(corpus_iterable=tqdm(tokens))
model_5.build_vocab(corpus_iterable=tqdm(tokens))
model_10.build_vocab(corpus_iterable=tqdm(tokens))

100%|██████████| 98570/98570 [00:00<00:00, 451299.26it/s]


모델의 vocabulary를 구축한다.

## train the model with 5 epochs

In [15]:
model_2.train(
    corpus_iterable=tokens,
    total_examples=model_2.corpus_count,
    epochs=5 
)

(5243961, 5646400)

In [23]:
model_5.train(
    corpus_iterable=tokens,
    total_examples=model_5.corpus_count, #train data set은 같으므로 vocab size는 같음
    epochs=5 
)

(5243910, 5646400)

In [20]:
model_10.train(
    corpus_iterable=tokens,
    total_examples=model_10.corpus_count, #train data set은 같으므로 vocab size는 같음
    epochs=5 
)

(5244389, 5646400)

window size가 클수록 훈련에 소요되는 시간이 길다. 

# 3. Embedding

## The embedding for the word “rain”

In [23]:
print(model_5.wv['rain'])

[-0.02645527  0.6395239  -0.4964388   0.10595167 -0.3906855  -0.53335226
  0.01744531 -0.12760144 -0.5144682   0.3725889  -0.10033483  0.3051476
 -0.35520017  0.5693141   0.40257344 -0.6281058   0.5690204  -0.53710884
 -0.16499928 -0.39856976 -0.34479478  0.00499978 -0.800955   -0.39290652
 -0.20052683 -0.6723169  -0.27447057 -0.22358577  0.43244877 -0.45949602
  0.21518448  0.26309088  0.06380791  0.09739598 -0.6712235   0.99001867
  0.5402711  -0.18458632  0.62612665 -0.11936793  1.4128401   0.06444914
  0.01482243 -0.24069378  0.26182976 -0.17467125 -0.56986034  0.44458035
  0.1963576   0.44392574 -0.5036081   0.51627296 -0.45226568  0.30621248
 -0.11523397 -0.28012472  0.7190037  -0.24569403  0.5464071  -0.4307584
  0.34494784  0.3148657  -0.8585322  -0.4157707  -0.2014352   0.14321333
  0.09888166  0.03536413 -0.02526478  0.58131725 -0.7631922  -0.53273433
  0.41895208  0.31111953  0.3952829   0.45690367  0.4151457   0.24656506
 -0.4499778   1.023306   -0.31272548  0.5954045  -0.0

## The top 5 similar words of the terms 'emma' and 'hamlet'

토큰화 과정에서 `.lower()`를 이용하여 모두 소문자로 token화 하였으므로 emma와 hamlet을 사용한다. 

### emma

In [27]:
print("window size 2 :", model_2.wv.most_similar(['emma'])[0:5],'\nwindow size 5 :', model_5.wv.most_similar(['emma'])[0:5], '\nwindow size 10 :', model_10.wv.most_similar(['emma'])[0:5])
      

window size 2 : [('volume', 0.7976555824279785), ('jane', 0.7858299016952515), ('ages', 0.7478381991386414), ('mrs', 0.7340356111526489), ('age', 0.7121933698654175)] 
window size 5 : [('elinor', 0.829072117805481), ('anne', 0.7928386330604553), ('cecilia', 0.7780994772911072), ('marianne', 0.7758905291557312), ('leonora', 0.765430748462677)] 
window size 10 : [('elinor', 0.9112785458564758), ('anne', 0.898774266242981), ('harriet', 0.8926092386245728), ('marianne', 0.8677136301994324), ('leonora', 0.8509418368339539)]


### hamlet

In [44]:
print("window size 2 :", model_2.wv.most_similar(['hamlet'])[0:5],'\nwindow size 5 :', model_5.wv.most_similar(['hamlet'])[0:5], '\nwindow size 10 :', model_10.wv.most_similar(['hamlet'])[0:5])

window size 2 : [('threat', 0.9929695129394531), ('dodo', 0.9909932017326355), ('sulk', 0.9909733533859253), ('strained', 0.9909714460372925), ('virginia', 0.9907004237174988)] 
window size 5 : [('queene', 0.8135713338851929), ('banquo', 0.8095588684082031), ('polonius', 0.793988049030304), ('deere', 0.7844197154045105), ('laertes', 0.7834975123405457)] 
window size 10 : [('sonne', 0.9150676727294922), ('banquo', 0.9080121517181396), ('marke', 0.9047688841819763), ('ophelia', 0.9039044976234436), ('queene', 0.901672899723053)]


윈도우 사이즈가 크다는 것은 반영하는 context가 많다는 것이다. 작은 윈도우 사이즈에서는 통사적 관련성이나 단어 자체의 쓰임이 유사한 단어가 나오며, 큰 윈도우 사이즈는 의미적 정보나 관계를 유사한 단어가 나온다. 위의 사례에서 윈도우사이즈가 작은 경우에는 의미적 연관을 유추할 수 없는 단어들이 출력되었지만, 5,10의 경우에는 소설의 등장인물일 것으로 추측되는 단어들이 출력되었다.

elinor 와 marianne는 emma와 같은 작가인 제인 오스틴의 다른 소설 Sense and Sensibility의 등장인물이며, anne는 persuation의 등장인물이다. harriet는 emma가 주인공인 소설 emma의 등장인물이다. 

(leonora와 cecilia는 제인오스틴의 소설이 아닌 다른 소설의 주인공이다.)

laertes, polonius ophelia는 햄릿의 등장인물이며, queene은 여왕이므로 모두 hamelt의 등장인물이다. sonne를 아들이란 뜻으로 추측한다면 왕자인 햄릿을 가리키는 것이라 생각할 수 있을것이다.
marke는 쉐익스피어의 희곡 Julius Caesar, banquo는 Macbet의 등장인물이다.

즉, emma는 같은 작가의 다른 책들의 등장인물과 유사도가 높으며, hamlet은 같은 책의 등장인물과의 유사도가 높다.

이는 쉐익스피어의 희곡으로 알려진 작품들이 공동저작물이라는 기사(출처 https://www.hani.co.kr/arti/culture/culture_general/249413.html)을 고려하면 word2vec모델을 작가의 스타일을 구별하는데 이용할 수 있지 않을까 생각할 수 있다. 

그런데 window size가 10였을 때는 쉐익스피어의 다른 작품의 등장인물이 등장한 것으로 보아그저 window size가 작아 local context만을 반영했기 때문일 수 있다.

# 4.  Compare models

In [1]:
# glove 모델 다운로드
import gensim.downloader as api
glove = api.load("glove-wiki-gigaword-100")

In [13]:
glove.most_similar(['emma'])[0:5]

[('emily', 0.7630564570426941),
 ('catherine', 0.7424191236495972),
 ('helen', 0.7261478900909424),
 ('kate', 0.7191192507743835),
 ('lucy', 0.7162103652954102)]

In [14]:
glove.most_similar(['hamlet'])[0:5]

[('village', 0.6998987197875977),
 ('town', 0.6558532118797302),
 ('situated', 0.5926076173782349),
 ('located', 0.5660547614097595),
 ('unincorporated', 0.5599358677864075)]

trained data가 다르니 두 모델의 성능 비교는 어려울 것이다. 가장 두드러지는 차이는 유사도 값이 `model`의 경우가 `glove`에 비해 상당히 높다는 것이다. 이는 trained data의 차이로 생각할수 있는데,  gutenberg의 경우 소설들로 구성된 텍스트이기 때문에 하나의 소설이라는 틀안에서 유사한 context를 가질 확률이 높기에 단어들의 벡터값이 비슷했을 것이다. 반면 `glove`는 위키피디아 및 뉴스를 학습시켰고 이들은 텍스트별로 주제가 상이하였을 것이기에 context가 유사할 확률이 낮다.
더불어 `model`에서 'emma', 'hamlet'과 유사한 단어는 작가나 같은 소설에서 등장했다는 점을 찾을 수 있지만, `glove`는 특정 주제에 연관되어있다기 보단 emma의 경우엔 영미권의 여자 이름 'hamlet'의 경우엔 지리에 관련된 단어들이 나타난 것을 
보았을 때 조금 더 보편적인 유사도를 지닌다고 할 수 있다. 

trained data의 차이 뿐 아니라 모델의 훈련방식에 있어서도 glove의 경우엔 윈도우 내부에서만이 아니라 전체적인 통계수치를 고려하기 때문에 general하게 나타난 것이라고도 생각해볼 수 있을 것이다.
또한 `model` 두 단어 그리고 `glove`의 'emma' 모두 사이즈가 큰 경우에 유사한 단어들이 명사였지만 hamlet의 경우에는 'situated', 'located', 'unincoportaed'와 같이 동사도 유사한 단어로 사용됨을 알 수 있다. 이는 word2vec의 훈련방식의 경우에는 window size가 크면 통사적 연관성이 무시될 수 있지만, `glove`의 경우에는 동시등장확률을 count하여 확률로 나타내기 때문에 통사적 정보도 반영할 수 있을 것이다.

# 5. Word similarity


In [4]:
print('similarity with apple', '\nboots :', glove.similarity(w1='apple', w2='boots'),'\nsteeve :', glove.similarity(w1='apple', w2='steve'), '\nplum :', glove.similarity(w1='apple', w2='plum'))


similarity with apple 
boots : 0.18325688 
steeve : 0.34688112 
plum : 0.5553858


코사인 유사도는 두 벡터의 사잇각을 (-1, 1)의 값으로 표현한 것으로 의미적 유사성을 나타낸다. 값이 1에 가까울 수록 의미적으로 유사함을 0에 가까울수록 의미적으로 관계가 없음을 -1에 가까울 수록 의미적으로 반대임을 의미한다. 'apple'과의 유사도는 'plum', 'steve', 'boots' 순으로 높았다. 의미적 유사도가 apple과 plum은 같은 과일이기 때문에 가장 높았을 것이다. steve와 apple은 값이 크진 않으니 의미적 유사성이 있다기 보단 어느정도 관련이 있다 정도만으로 해석할 수 있을 것이며, apple과 boots는 관련이 없다고 할 수 있다. 이것도 결국은 train data가 되는 corpus 내에서의 상대적인 유사성임을 유념해야한다.

아래의 코드에서 word2vec 방식에 gutenberg.txt 파일로 훈련시킨 `model`의 유사도를 보면 'phil'보다 'boots'가 'apple'과의 유사도가 높은 것을 확인할 수 있다.(사실이는 정확한 비교는 될 수 없는 것이 `model`의 vocab에 'steve'가 없기에 `glove`에서 'steve'와 유사한 'phil'과 'apple'의 유사도를 측정하였기 때문이다.)

In [5]:
glove.most_similar(['steve'])[0]

('phil', 0.7983406186103821)

In [24]:
print('similarity with apple', '\nboots :', model_5.wv.similarity(w1='apple', w2='boots'),'\nphil :', model_5.wv.similarity(w1='apple', w2='phil'), '\nplum :', model_5.wv.similarity(w1='apple', w2='plum'))


similarity with apple 
boots : 0.7332332 
steeve : 0.7320809 
plum : 0.85719514
